# Installing Required Packages

In [1]:
!pip3.10 install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip3.10 install psycopg2

Defaulting to user installation because normal site-packages is not writeable


# Importing the Required Packages

In [1]:
#Importing Package
import sqlalchemy
#Database Utility Class
from sqlalchemy.engine import create_engine
# Provides executable SQL expression construct
from sqlalchemy.sql import text
sqlalchemy.__version__ 

'1.4.39'

In [2]:
class PostgresqlDB:
    def __init__(self,user_name,password,host,port,db_name):
        """
        class to implement DDL, DQL and DML commands,
        user_name:- username
        password:- password of the user
        host
        port:- port number
        db_name:- database name
        """
        self.user_name = user_name
        self.password = password
        self.host = host
        self.port = port
        self.db_name = db_name
        self.engine = self.create_db_engine()

    def create_db_engine(self):
        """
        Method to establish a connection to the database, will return an instance of Engine
        which can used to communicate with the database
        """
        try:
            db_uri = f"postgresql+psycopg2://{self.user_name}:{self.password}@{self.host}:{self.port}/{self.db_name}"
            return create_engine(db_uri)
        except Exception as err:
            raise RuntimeError(f'Failed to establish connection -- {err}') from err

    def execute_dql_commands(self,stmnt,values=None):
        """
        DQL - Data Query Language
        SQLAlchemy execute query by default as 

        BEGIN
        ....
        ROLLBACK 

        BEGIN will be added implicitly everytime but if we don't mention commit or rollback explicitly 
        then rollback will be appended at the end.
        We can execute only retrieval query with above transaction block.If we try to insert or update data 
        it will be rolled back.That's why it is necessary to use commit when we are executing 
        Data Manipulation Langiage(DML) or Data Definition Language(DDL) Query.
        """
        try:
            with self.engine.connect() as conn:
                if values is not None:
                    result = conn.execute(text(stmnt),values)
                else:
                    result = conn.execute(text(stmnt))
            return result
        except Exception as err:
            print(f'Failed to execute dql commands -- {err}')
    
    def execute_ddl_and_dml_commands(self,stmnt,values=None):
        """
        Method to execute DDL and DML commands
        here we have followed another approach without using the "with" clause
        """
        connection = self.engine.connect()
        trans = connection.begin()
        try:
            if values is not None:

                result = connection.execute(text(stmnt),values)
            else:
                result = connection.execute(text(stmnt))
            trans.commit()
            connection.close()
            print('Command executed successfully.')
        except Exception as err:
            trans.rollback()
            print(f'Failed to execute ddl and dml commands -- {err}')

In [3]:
#Defining Db Credentials
USER_NAME = 'postgres'
PASSWORD = 'postgres'
PORT = 5432
DATABASE_NAME = 'ems'
HOST = 'localhost'

#Note - Database should be created before executing below operation
#Initializing SqlAlchemy Postgresql Db Instance
db = PostgresqlDB(user_name=USER_NAME,
                    password=PASSWORD,
                    host=HOST,port=PORT,
                    db_name=DATABASE_NAME)
engine = db.engine

In [6]:
roles = "CREATE TABLE roles(id integer PRIMARY KEY, dept text not null, \
    name text not null, role_level integer not null);"
db.execute_ddl_and_dml_commands(roles)

Command executed successfully.


In [7]:
employee = "CREATE TABLE employee(id integer PRIMARY KEY, \
    first_name text not null, \
    last_name text not null, \
    dob date not null, \
    ph_num bigint not null, gender char(1) not null, \
    email text not null, join_date date not null, \
    address text, account_num bigint not null, \
    working_currently boolean not null);"
db.execute_ddl_and_dml_commands(employee)

Command executed successfully.


In [8]:
deductions = "CREATE TABLE deductions(id serial PRIMARY KEY, role_id integer, name text not null, amount integer not null, issue_date date not null, empid integer, \
    constraint fk_deductions_emp foreign key (empid) references employee(id), \
    constraint fk_deductions_role foreign key (role_id) references roles(id));"
db.execute_ddl_and_dml_commands(deductions)

Command executed successfully.


In [9]:
clients = "CREATE TABLE clients(id integer PRIMARY KEY, name text not null, \
    institution text, ph_num bigint not null, email text not null);"
db.execute_ddl_and_dml_commands(clients)

Command executed successfully.


In [10]:
projects = "CREATE TABLE projects(id integer PRIMARY KEY, \
    main_dept text, proj_head_one integer , \
    proj_head_two integer, start_date date not null, \
    status text not null, time_taken integer, \
    proj_type text, constraint fk_ph_one_emp foreign key (proj_head_one) \
    references employee(id), constraint fk_ph_two_emp foreign key (proj_head_two) \
    references employee(id));"
db.execute_ddl_and_dml_commands(projects)

Command executed successfully.


In [11]:
clientproject = "CREATE TABLE clientproject(proj_id integer, client_id integer, \
    feedback integer, review text, \
    PRIMARY KEY (proj_id, client_id), constraint fk_proj_cp foreign key (proj_id)\
    references projects(id), constraint fk_client_cp foreign key (client_id) \
    references clients(id));"
db.execute_ddl_and_dml_commands(clientproject)

Command executed successfully.


In [12]:
allowances = "CREATE TABLE allowances(id integer PRIMARY KEY, role_id integer, \
    name text not null, amount integer not null, frequency integer not null, \
    year integer not null, empid integer, \
    constraint fk_allowance_emp foreign key (empid) references employee(id), \
    constraint fk_allowance_role foreign key (role_id) references roles(id));"
db.execute_ddl_and_dml_commands(allowances)

Command executed successfully.


In [13]:
exemployee = "CREATE TABLE exemployee(id integer PRIMARY KEY, \
    date_left date not null, \
    reason varchar(20), constraint fk_emp_exemp foreign key (id) \
    references employee(id));"
db.execute_ddl_and_dml_commands(exemployee)

Command executed successfully.


In [14]:
leaves = "CREATE TABLE leaves( leave_type integer not null, \
    role_id integer, allowed_num integer not null, \
    pay_reduct integer, empid integer, \
    num_taken integer not null, taken_date date not null, \
    PRIMARY KEY (empid, taken_date), \
    constraint fk_leaves_emp foreign key (empid) references \
    employee(id), constraint fk_leaves_role foreign key \
    (role_id) references roles(id));"
db.execute_ddl_and_dml_commands(leaves)

Command executed successfully.


In [15]:
logindetails = "CREATE TABLE logindetails(empid integer, login_date date, \
    intime time not null, outtime time not null, \
    PRIMARY KEY (empid, login_date), \
    constraint fk_emp_login foreign key (empid) \
    references employee(id));"
db.execute_ddl_and_dml_commands(logindetails)

Command executed successfully.


In [16]:
performancescore = "CREATE TABLE performancescore(emp_id integer, proj_id integer, \
    role_id integer, performance integer, \
    PRIMARY KEY (emp_id, proj_id, role_id), \
    constraint fk_emp_ps foreign key (emp_id) references employee(id), \
    constraint fk_proj_ps foreign key (proj_id) references projects(id), \
    constraint fk_role_ps foreign key (role_id) references roles(id));"
db.execute_ddl_and_dml_commands(performancescore)

Command executed successfully.


In [17]:
salary = "CREATE TABLE salary(empid integer, issue_date date, basic_pay integer not null, gross integer not null, net_salary integer not null, performance_bonus integer not null, PRIMARY KEY(empid, issue_date), \
    constraint fk_salary_emp foreign key (empid) references employee(id) );"
db.execute_ddl_and_dml_commands(salary)

Command executed successfully.


In [18]:
skills = "CREATE TABLE skills(name_skill text not null, emp_id integer , PRIMARY KEY(name_skill, emp_id),\
    constraint fk_emp_skill foreign key (emp_id) \
    references employee(id) );"
db.execute_ddl_and_dml_commands(skills)

Command executed successfully.


In [19]:
training = "CREATE TABLE training(id serial PRIMARY KEY, name_skill text not null, tenure integer not null, \
    empid integer , constraint fk_emp_training foreign key (empid) \
    references employee(id));"
db.execute_ddl_and_dml_commands(training)

Command executed successfully.


In [100]:
transactions = "CREATE TABLE transactions(id bigint PRIMARY KEY, empid integer, transfer_date date not null, amount integer, status text not null, \
    constraint fk_emp_transfer foreign key (empid) references employee(id));"
db.execute_ddl_and_dml_commands(transactions)

Command executed successfully.


In [21]:
import pandas as pd
import numpy

In [22]:
ALLOWANCES = pd.read_csv('./data/Allowances.csv', header=0, skiprows=[1])
DEDUCTIONS = pd.read_csv('./data/Deductions.csv', header=0, skiprows=[1])
CLIENTS = pd.read_csv('./data/Clients.csv', header=0, skiprows=[1])
CLIENTPROJECT = pd.read_csv('./data/client-project.csv', header=0, skiprows=[1])
EMPLOYEE = pd.read_csv('./data/Employee.csv', header=0, skiprows=[1])
EXEMPLOYEE = pd.read_csv('./data/Ex-employee.csv', header=0, skiprows=[1])
LEAVES = pd.read_csv('./data/Leaves.csv', header=0, skiprows=[1])
LOGINDETAILS = pd.read_csv('./data/Login-Details.csv', header=0, skiprows=[1])
PERFORMANCESCORE = pd.read_csv('./data/Performance-Score.csv', header=0, skiprows=[1])
PROJECTS = pd.read_csv('./data/Projects.csv', header=0, skiprows=[1])
ROLES = pd.read_csv('./data/Roles.csv', header=0, skiprows=[1])
SALARY = pd.read_csv('./data/Salary.csv', header=0, skiprows=[1])
SKILLS = pd.read_csv('./data/Skills.csv', header=0, skiprows=[1])
TRAINING = pd.read_csv('./data/Training.csv', header=0, skiprows=[1])
TRANSACTIONS = pd.read_csv('./data/Transactions.csv', header=0, skiprows=[1])

In [23]:
CLIENTS['Phone number'] = CLIENTS['Phone number'].str.replace("-","")

In [24]:
PROJECTS['Time taken'] = PROJECTS['Time taken'].str.replace("(null)","")

In [25]:
CLIENTPROJECT['Feedback'] = CLIENTPROJECT['Feedback'].astype(str)
CLIENTPROJECT['Feedback'] = CLIENTPROJECT['Feedback'].str.replace(".0","")

In [26]:
PERFORMANCESCORE['Performance score'] = PERFORMANCESCORE['Performance score'].astype(str).str.replace(".0","")

In [27]:
EMPLOYEE[' first name '] = EMPLOYEE[' first name '].str.strip()
EMPLOYEE[' last name '] = EMPLOYEE[' last name '].str.strip()
EMPLOYEE[' phone number '] = EMPLOYEE[' phone number '].str.strip()
EMPLOYEE[' phone number '] = EMPLOYEE[' phone number '].str.replace("-", "")
EMPLOYEE[' email '] = EMPLOYEE[' email '].str.strip()
EMPLOYEE[' address '] = EMPLOYEE[' address '].str.strip()
EMPLOYEE[' gender '] = EMPLOYEE[' gender '].str.strip()
print(EMPLOYEE.head())

   employee id   first name   last name   date of birth    role level    
0             1         John       Smith      1985-05-10             3  \
1             2         Jane         Doe      1990-08-15             2   
2             3      Michael     Johnson      1992-01-22             1   
3             4        Sarah    Williams      1988-04-27             4   
4             5        David       Brown      1995-11-02             2   

   phone number   gender                        email   joining date    
0        5551234        M       john.smith@example.com     2015-01-01  \
1        5555678        F         jane.doe@example.com     2016-02-01   
2        5559876        M  michael.johnson@example.com     2017-03-01   
3        5554321        F   sarah.williams@example.com     2018-04-01   
4        5558765        M      david.brown@example.com     2019-05-01   

                     address    bank account number    working currently  
0   123 Main St, Anytown, USA            

In [28]:
for x in ROLES.index:
    entry = {'id':int(ROLES['Role id'][x]), 'dept':ROLES['Department'][x], 'name':ROLES['Name'][x], 'role_level':int(ROLES['Role level'][x])}
    insert = "INSERT INTO roles (id, dept, name, role_level) VALUES \
        (:id, :dept, :name, :role_level);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [29]:
for x in EMPLOYEE.index:
    entry = {'id':int(EMPLOYEE['employee id '][x]), 'first_name':EMPLOYEE[' first name '][x], 'last_name':EMPLOYEE[' last name '][x], 'dob':EMPLOYEE[' date of birth '][x], 'ph_num':int(EMPLOYEE[' phone number '][x]), 'gender':EMPLOYEE[' gender '][x], 'email':EMPLOYEE[' email '][x], 'address':EMPLOYEE[' address '][x], 'join_date':EMPLOYEE[' joining date '][x], 'account_num':int(EMPLOYEE[' bank account number '][x]), 'working_currently':bool(EMPLOYEE[' working currently'][x])}
    insert = "INSERT INTO employee (id, first_name, last_name, dob, ph_num, gender, email, address, join_date, account_num, working_currently) \
        VALUES (:id, :first_name, :last_name, :dob, :ph_num, :gender, :email, :address, :join_date, :account_num, :working_currently);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [30]:
for x in DEDUCTIONS.index:
    entry = {'id':int(DEDUCTIONS['Deduction id'][x]), 'role_id':int(DEDUCTIONS['Role id'][x]), 'name':DEDUCTIONS['Name'][x], 'amount':int(DEDUCTIONS['Amount'][x]), 'issue_date':DEDUCTIONS['Date'][x], 'empid':int(DEDUCTIONS['Employee id'][x])}
    insert = "INSERT INTO deductions (id, role_id, name, amount, issue_date, empid) VALUES \
        (:id, :role_id, :name, :amount, :issue_date, :empid);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [31]:
for x in CLIENTS.index:
    entry = {'id':int(CLIENTS['Client id'][x]), 'name':CLIENTS['Name'][x], 'institution':CLIENTS['Institution'][x], 'ph_num':int(CLIENTS['Phone number'][x]), 'email':CLIENTS['Email id'][x]}
    insert = "INSERT INTO clients (id, name, institution, ph_num, email) VALUES \
        (:id, :name, :institution, :ph_num, :email);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [32]:
for x in PROJECTS.index:
    if(PROJECTS['Time taken'][x] == ""):
        entry = {'id':int(PROJECTS['Project id'][x]), 'main_dept':PROJECTS['Main Department'][x], 'proj_head_one':int(PROJECTS['Project head 1'][x]), 'proj_head_two':int(PROJECTS['Project head 2'][x]), 'start_date':PROJECTS['Start date'][x], 'status':PROJECTS['Status'][x], 'proj_type':PROJECTS['Project type'][x]}
        insert = "INSERT INTO projects (id, main_dept, proj_head_one, proj_head_two, start_date, status, proj_type) \
            VALUES (:id, :main_dept, :proj_head_one, :proj_head_two, :start_date, :status, :proj_type);"
    else:
        entry = {'id':int(PROJECTS['Project id'][x]), 'main_dept':PROJECTS['Main Department'][x], 'proj_head_one':int(PROJECTS['Project head 1'][x]), 'proj_head_two':int(PROJECTS['Project head 2'][x]), 'start_date':PROJECTS['Start date'][x], 'status':PROJECTS['Status'][x], 'time_taken':int(PROJECTS['Time taken'][x]), 'proj_type':PROJECTS['Project type'][x]}
        insert = "INSERT INTO projects (id, main_dept, proj_head_one, proj_head_two, start_date, status, time_taken, proj_type) \
            VALUES (:id, :main_dept, :proj_head_one, :proj_head_two, :start_date, :status, :time_taken, :proj_type);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [33]:
for x in CLIENTPROJECT.index:
    if(CLIENTPROJECT['Feedback'][x] == "nan"):
        entry = {'proj_id':int(CLIENTPROJECT['Project id'][x]), 'client_id':int(CLIENTPROJECT['Client id'][x])}
        insert = "INSERT INTO clientproject (proj_id, client_id) VALUES \
        (:proj_id, :client_id);"
    else:
        entry = {'proj_id':int(CLIENTPROJECT['Project id'][x]), 'client_id':int(CLIENTPROJECT['Client id'][x]), 'feedback':int(CLIENTPROJECT['Feedback'][x]), 'review':CLIENTPROJECT['Review Note'][x]}
        insert = "INSERT INTO clientproject (proj_id, client_id, feedback, review) VALUES \
            (:proj_id, :client_id, :feedback, :review);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [34]:
for x in ALLOWANCES.index:
    entry = {'id':int(ALLOWANCES['Allowance id'][x]), 'role_id':int(ALLOWANCES['Role id'][x]), 'name':ALLOWANCES['Name'][x], 'amount':int(ALLOWANCES['Amount'][x]), 'frequency':int(ALLOWANCES['Frequency_per_year'][x]), 'year':int(ALLOWANCES['Year'][x]), 'empid':int(ALLOWANCES['employee id'][x])}
    insert = "INSERT INTO allowances (id, role_id, name, amount, frequency, year, empid) VALUES \
        (:id, :role_id, :name, :amount, :frequency, :year, :empid);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [35]:
for x in EXEMPLOYEE.index:
    entry = {'id':int(EXEMPLOYEE['Employee id'][x]), 'date_left':EXEMPLOYEE['Date of leaving'][x], 'reason':EXEMPLOYEE['Reason/Remarks'][x]}
    insert = "INSERT INTO exemployee (id, date_left, reason) VALUES \
        (:id, :date_left, :reason);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.


In [36]:
for x in LEAVES.index:
    entry = {'leave_type':int(LEAVES['Leave type'][x]), 'role_id':int(LEAVES['Role id'][x]), 'allowed_num':int(LEAVES['Allowed num'][x]), 'pay_reduct':int(LEAVES['Payment reduction per leave taken'][x]), 'empid':int(LEAVES['Employee id'][x]), 'num_taken':int(LEAVES['Number of leaves taken'][x]), 'taken_date':LEAVES['Date'][x]}
    insert = "INSERT INTO leaves (leave_type, role_id, allowed_num, pay_reduct, empid, num_taken, taken_date) VALUES \
        (:leave_type, :role_id, :allowed_num, :pay_reduct, :empid, :num_taken, :taken_date);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [37]:
for x in LOGINDETAILS.index:
    entry = {'empid':int(LOGINDETAILS['Employee id'][x]), 'login_date':LOGINDETAILS['Date'][x], 'intime':LOGINDETAILS['In Time'][x], 'outtime':LOGINDETAILS['Out Time'][x]}
    insert = "INSERT INTO logindetails (empid, login_date, intime, outtime) VALUES \
        (:empid, :login_date, :intime, :outtime);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [38]:
for x in PERFORMANCESCORE.index:
    if (PERFORMANCESCORE['Performance score'][x] == "nan"):
        entry = {'emp_id':int(PERFORMANCESCORE['Employee id'][x]), 'proj_id':int(PERFORMANCESCORE['Project id'][x]), 'role_id':int(PERFORMANCESCORE['Role id'][x])}
        insert = "INSERT INTO performancescore (emp_id, proj_id, role_id) VALUES \
            (:emp_id, :proj_id, :role_id)"
    else:
        entry = {'emp_id':int(PERFORMANCESCORE['Employee id'][x]), 'proj_id':int(PERFORMANCESCORE['Project id'][x]), 'role_id':int(PERFORMANCESCORE['Role id'][x]), 'performance':int(PERFORMANCESCORE['Performance score'][x])}
        insert = "INSERT INTO performancescore (emp_id, proj_id, role_id, performance) VALUES \
            (:emp_id, :proj_id, :role_id, :performance)"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [63]:
for x in SALARY.index:
    entry = {'empid':int(SALARY['Employee id'][x]), 'issue_date':SALARY['Date'][x], 'basic_pay':int(SALARY['Basic pay'][x]), 'gross':0, 'net_salary':0, 'performance_bonus':int(SALARY['Performance Bonus'][x])}
    insert = "INSERT INTO salary (empid, issue_date, basic_pay, gross, net_salary, performance_bonus) VALUES \
        (:empid, :issue_date, :basic_pay, :gross, :net_salary, :performance_bonus);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [40]:
for x in SKILLS.index:
    entry = {'name_skill':SKILLS['Name'][x], 'emp_id':int(SKILLS['Employee id'][x])}
    insert = "INSERT INTO skills (name_skill, emp_id) VALUES \
        (:name_skill, :emp_id);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [41]:
for x in TRAINING.index:
    entry = {'id': int(TRAINING['Training id'][x]), 'name_skill':TRAINING['Domain'][x], 'tenure':int(TRAINING['Tenure'][x]), 'empid':int(TRAINING['Employee id'][x])}
    insert = "INSERT INTO training (id, name_skill, tenure, empid) VALUES \
        (:id, :name_skill, :tenure, :empid);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.


In [112]:
for x in TRANSACTIONS.index:
    entry = {'id':int(TRANSACTIONS['Transaction id'][x]), 'empid':int(TRANSACTIONS['Employee id'][x]), 'transfer_date':TRANSACTIONS['Date'][x], 'amount':0, 'status':TRANSACTIONS['Status'][x]}
    insert = "INSERT INTO transactions (id, empid, transfer_date, amount, status) VALUES \
        (:id, :empid, :transfer_date, :amount, :status);"
    db.execute_ddl_and_dml_commands(insert, entry)

Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command executed successfully.
Command 

In [62]:
query = "create or replace function check_salary() \
returns trigger \
language plpgsql \
as $$ \
declare \
	r record; \
	last_date date; \
	total_alwns integer; \
	total_ddcts integer; \
begin \
	if (new.empid in (select id from exemployee)) then \
		raise notice 'Entry for ex-employee given'; \
	else \
	end if; \
	select issue_date into last_date from salary where empid = new.empid order by issue_date desc limit 1; \
	if((new.issue_date - last_date) < 28) then \
		raise notice 'Salary being given more than once'; \
	else \
	end if; \
	total_alwns = 0; \
	total_ddcts = 0; \
	for r in select amount, frequency from allowances where empid = new.empid AND year = EXTRACT(YEAR FROM new.issue_date) \
	loop \
		if(r.frequency=1) then \
			if(EXTRACT(MONTH FROM new.issue_date) = 12 AND EXTRACT(DAY FROM new.issue_date) > 15) then \
				total_alwns = total_alwns + r.amount; \
			else \
			end if; \
		else \
			if(r.frequency=4) then \
				if((EXTRACT(MONTH FROM new.issue_date) = 12 OR EXTRACT(MONTH FROM new.issue_date) = 9 OR EXTRACT(MONTH FROM new.issue_date) = 3 OR EXTRACT(MONTH FROM new.issue_date) = 6) AND EXTRACT(DAY FROM new.issue_date) > 15) then \
					total_alwns = total_alwns + r.amount; \
				else \
				end if; \
			else \
				if(r.frequency=6) then \
					if((EXTRACT(MONTH FROM new.issue_date) = 12 OR EXTRACT(MONTH FROM new.issue_date) = 6) AND EXTRACT(DAY FROM new.issue_date) > 15) then \
						total_alwns = total_alwns + r.amount; \
					else \
					end if; \
				else \
					if(EXTRACT(day FROM new.issue_date) > 15) then \
						total_alwns = total_alwns + r.amount; \
					else \
					end if; \
				end if; \
			end if; \
		end if; \
	end loop; \
	for r in select amount from deductions where empid = new.empid AND issue_date > last_date AND issue_date <= new.issue_date \
	loop \
		total_ddcts = total_ddcts + r.amount; \
	end loop; \
	new.gross = new.basic_pay + total_alwns; \
	new.net_salary = new.gross - total_ddcts; \
	if(new.net_salary < 0) then \
		raise notice 'Invalid salary amount'; \
	else \
	end if; \
	return new; \
end; \
$$;"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [47]:
query = "create trigger check_salary_trigger \
before update or insert \
on salary \
for each row \
execute procedure check_salary(); \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [113]:
query = "create or replace function check_transactions() \
returns trigger \
language plpgsql \
as $$ \
declare \
	amt integer; \
begin \
	if (new.empid in (select id from exemployee)) then \
		raise notice 'Entry for ex-employee given'; \
	else \
	end if; \
	if (0 = 0) then \
		select net_salary + performance_bonus into amt from salary where empid = new.empid AND new.transfer_date = '2022-01-03'; \
		if (amt != new.amount AND new.amount IS NOT NULL) then \
			new.amount = amt; \
		else \
		end if; \
		new.amount = amt; \
		return new; \
	else \
		raise notice 'Invalid transaction'; \
	end if; \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [ ]:
query = "create trigger check_transactions_trigger \
before update or insert \
on transactions \
for each row \
execute procedure check_transactions(); \
"
db.execute_ddl_and_dml_commands(query)

In [119]:
query = "create or replace function client_project_details(type text) \
returns table (proj_id integer, proj_dept text, ph_name_one text, ph_ph_num_one bigint, ph_email_one text, ph_name_two text, ph_ph_num_two bigint, ph_email_two text, proj_status text) \
language plpgsql \
as $$ \
declare  \
	r record; \
	emp_name text; \
	emp_phnum bigint; \
	emp_email text; \
begin \
	for r in (select * from projects where proj_type ILIKE type) \
loop \
	proj_id := r.id; \
	proj_dept := r.main_dept; \
	proj_status := r.status; \
	select e.first_name || ' ' || e.last_name into emp_name from employee e where e.id = r.proj_head_one; \
	select e.ph_num into emp_phnum from employee e where e.id = r.proj_head_one; \
	select e.email into emp_email from employee e where e.id = r.proj_head_one; \
	ph_name_one := emp_name; \
	ph_ph_num_one := emp_phnum; \
	ph_email_one := emp_email; \
	select e.first_name || ' ' || e.last_name into emp_name from employee e where e.id = r.proj_head_two; \
	select e.ph_num into emp_phnum from employee e where e.id = r.proj_head_two; \
	select e.email into emp_email from employee e where e.id = r.proj_head_two; \
	ph_name_two := emp_name; \
	ph_ph_num_two := emp_phnum; \
	ph_email_two := emp_email; \
	return next; \
end loop;  \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [147]:
query = "create or replace function employee_performance() \
returns table (id integer, proj_done integer, total_score integer, ongoing integer) \
language plpgsql \
as $$ \
begin \
    return query \
	select e.id, c.proj_done::integer, c.total_score::integer, c.ongoing::integer \
from (select o.emp_id, coalesce(j.proj_done,0) as proj_done, coalesce(j.total_score,0) as total_score,  o.total_proj - coalesce(j.proj_done,0) as ongoing \
from  \
(select ps.emp_id,COUNT(ps.proj_id) as proj_done, SUM(ps.performance) as total_score  \
from performancescore ps join projects p  \
on ps.proj_id = p.id  where p.status = 'Completed' group by ps.emp_id) as j  \
right join  \
(select emp_id, COUNT(proj_id) as total_proj  \
from performancescore group by emp_id ) as o  \
on o.emp_id = j.emp_id) as c right join employee e  \
on e.id = c.emp_id  \
where e.working_currently = true ; \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [4]:
query = "create or replace function employee_project_allocate(skill_name text) \
returns table (id integer, proj_done integer, total_score integer, ongoing integer) \
language plpgsql \
as $$ \
begin \
    return query \
	select e.id, c.proj_done::integer, c.total_score::integer, c.ongoing::integer \
from (select t.emp_id, t.proj_done, t.total_score, t.ongoing \
	from skills s join \
(select o.emp_id, coalesce(j.proj_done,0) as proj_done, coalesce(j.total_score,0) as total_score,  o.total_proj - coalesce(j.proj_done,0) as ongoing \
from \
(select ps.emp_id,COUNT(ps.proj_id) as proj_done, SUM(ps.performance) as total_score  \
from performancescore ps join projects p \
on ps.proj_id = p.id AND p.status = 'Completed' group by ps.emp_id) as j  \
right join \
(select emp_id, COUNT(proj_id) as total_proj  \
from performancescore group by emp_id ) as o \
on o.emp_id = j.emp_id) as t \
on t.emp_id = s.emp_id AND s.name_skill = skill_name) as c join employee e  \
on e.id = c.emp_id \
where e.working_currently = true \
order by c.ongoing desc; \
end; \
$$;"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [5]:
query = "create or replace function performance_bonus_calculation(project integer, bonus_per_score integer) \
returns table (id integer, bonus integer) \
language plpgsql \
as $$ \
begin \
    return query \
	select emp_id, performance*bonus_per_score  \
from performancescore \
where proj_id = project; \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [6]:
query = "create or replace function search_review(word text) \
returns table (id integer, cnt integer) \
language plpgsql \
as $$ \
begin \
    return query \
	select proj_id, COUNT(client_id)::integer  \
	from clientproject \
	where review ILIKE '%' || word || '%'\
	group by proj_id; \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [7]:
query = "create or replace function calculate_allowance_login_details(start_date date, end_date date, amt_per_hr integer) \
returns table (id integer, amount integer) \
language plpgsql \
as $$ \
declare \
    r record; \
    s record; \
	cnt integer; \
    hrs integer; \
begin \
	for r in (select e.id from employee e where e.working_currently = true) \
	loop \
		cnt = 0; \
		for s in (select login_date, intime, outtime from logindetails where empid = r.id) \
		loop \
			if (s.login_date BETWEEN start_date AND end_date) then \
				hrs = EXTRACT(HOURS FROM s.outtime - s.intime ) - 7; \
				if (hrs >= 1) then \
					cnt = cnt + hrs; \
				end if; \
			else \
			end if; \
		end loop; \
		id := r.id; \
		amount := cnt*amt_per_hr; \
		return next; \
	end loop; \
end; \
$$; \
"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.


In [8]:
query = "create or replace function employee_project_allocation(projid integer, roleid integer, skill_name text, min_projs integer) \
returns void \
language plpgsql \
as $$ \
declare \
    empid integer; \
begin \
    empid = 0; \
	select e.id into empid \
from (select t.emp_id, t.proj_done, t.total_score, t.ongoing \
	from skills s join \
(select o.emp_id, coalesce(j.proj_done,0) as proj_done, coalesce(j.total_score,0) as total_score,  o.total_proj - coalesce(j.proj_done,0) as ongoing \
from \
(select ps.emp_id,COUNT(ps.proj_id) as proj_done, SUM(ps.performance) as total_score  \
from performancescore ps join projects p \
on ps.proj_id = p.id AND p.status = 'Completed' group by ps.emp_id) as j  \
right join \
(select emp_id, COUNT(proj_id) as total_proj  \
from performancescore group by emp_id ) as o \
on o.emp_id = j.emp_id) as t \
on t.emp_id = s.emp_id AND s.name_skill = skill_name) as c join employee e  \
on e.id = c.emp_id \
where e.working_currently = true and c.ongoing >= min_projs \
order by c.total_score desc \
limit 1; \
if (empid = 0) then \
    raise notice 'Employee with requirements does not exist'; \
else \
    INSERT INTO performancescore (emp_id, proj_id, role_id) VALUES (empid, projid, roleid); \
end if; \
end; \
$$;"
db.execute_ddl_and_dml_commands(query)

Command executed successfully.
